In [85]:
import pandas as pd
import numpy as np
import os

In [86]:
cwd = os.getcwd()

In [87]:
fp = os.path.join(cwd,'Package_122816/abcd_ant01.txt')

df = pd.read_csv(fp,sep='\t')
df.drop(index=0,inplace=True)
num_cols = len(list(df.columns[9:-2]))

In [88]:
# Since we only want to take the earliest record for each subject, we need to
# 1. First convert the 'interview_date' column to a pandas DateTime type
df['interview_date'] = pd.to_datetime(df['interview_date'])

# Next, we need to remove the redundant entries for each subject
# 1. order the records chronologically (in order of early --> later)
df_1 = df.sort_values(by='interview_date')

# 2. Drop duplicate records (based on subjectkey), keeping the first occurance (i.e. temporally first, due to chron. ordering)
df_1 = df_1.drop_duplicates(subset='subjectkey',keep='first')

In [89]:
df_1.shape

(11875, 23)

In [90]:
# Next, convert values to numeric and categorical
categorical_cols = []
# Convert columns to numeric values where possible
# Keep track of cols that fail so they can be made into categorical data
for col in df_1.columns[9:-2]:
    try:
        df_1[col] = pd.to_numeric(df_1[col], errors='raise')
    except ValueError as e:
        print(e)
        print("1 could not convert column {}\n".format(col))
        categorical_cols.append(col)

print(categorical_cols)

# Following line needed before categorical casting
df_1[categorical_cols] = df_1[categorical_cols].fillna(value='MISSING')

# # Now, try to convert remaining columns to categories (only the ones which couldnt convert to numeric)
for col in df_1.select_dtypes(exclude=['number']).columns[9:-2]:
    # for col in categorical:
    try:
        df_1[col] = pd.Categorical(df_1[col], categories=df_1[col].unique()).codes
    except ValueError as e:
        print(e)
        print("2 could not convert column {}\n".format(col))

# print(df.dtypes)
# df

Unable to parse string "Arm" at position 339
1 could not convert column anthro_weight_a_location

Unable to parse string "2017-07-24 19:18" at position 1
1 could not convert column anthro_timestamp

['anthro_weight_a_location', 'anthro_timestamp']


In [91]:
df.dtypes

collection_id                       object
abcd_ant01_id                       object
dataset_id                          object
subjectkey                          object
src_subject_id                      object
interview_date              datetime64[ns]
interview_age                       object
gender                              object
eventname                           object
anthro_1_height_in                  object
anthro2heightin                     object
anthro3heightin                     object
anthroheightcalc                    object
anthroweightcast                    object
anthro_weight_a_location            object
anthroweight1lb                     object
anthroweight2lb                     object
anthroweight3lb                     object
anthroweightcalc                    object
anthro_waist_cm                     object
anthro_timestamp                    object
collection_title                    object
study_cohort_name                   object
dtype: obje

In [92]:
badcols=[]  # List to store the names of columns where either >50% data missing, or >95% values the same
# Iterate over the columns and check these crieria, store bad columns in the badcols list

num_rows = df_1.shape[0]

for col in df_1.columns[9:-2]:
    tmp = list(df_1[col].value_counts())  # find size of equal-values groups (ignoring NaNs)
    print(col)
    print(tmp[0])
    largest_eq_val_group = tmp[0]               # size of largest equal-value group
    num_nan = df_1[col].isnull().sum()            # Count how many NaNs there are
    num_not_nan = df_1[col].count()               # Count how many rows do NOT have NaNs

    print("COLUMN: {}\n".format(col))
    print('Largest equal vals group:\t{}\nnum nan values:\t{}\nnum not nan values:\t{}\n'.format(largest_eq_val_group, num_nan, num_not_nan))

    # 1. more than 50% missing?
    if ( (num_nan/num_rows)>0.5 ):
        badcols.append(col)
        continue
    # 2. largest equal-values group is >95% of entries?
    elif ( (largest_eq_val_group/num_not_nan)>0.95 ):
        badcols.append(col)

print(badcols)

# Drop the bad columns from the dataframe
df_2 = df_1.drop(columns=badcols)


anthro_1_height_in
649
COLUMN: anthro_1_height_in

Largest equal vals group:	649
num nan values:	11
num not nan values:	11864

anthro2heightin
643
COLUMN: anthro2heightin

Largest equal vals group:	643
num nan values:	9
num not nan values:	11866

anthro3heightin
16
COLUMN: anthro3heightin

Largest equal vals group:	16
num nan values:	11491
num not nan values:	384

anthroheightcalc
594
COLUMN: anthroheightcalc

Largest equal vals group:	594
num nan values:	9
num not nan values:	11866

anthroweightcast
11321
COLUMN: anthroweightcast

Largest equal vals group:	11321
num nan values:	527
num not nan values:	11348

anthro_weight_a_location
11850
COLUMN: anthro_weight_a_location

Largest equal vals group:	11850
num nan values:	0
num not nan values:	11875

anthroweight1lb
192
COLUMN: anthroweight1lb

Largest equal vals group:	192
num nan values:	11
num not nan values:	11864

anthroweight2lb
188
COLUMN: anthroweight2lb

Largest equal vals group:	188
num nan values:	12
num not nan values:	11863


In [94]:
df_2.shape

(11875, 19)